In [6]:
# comparison_csv.ipynb

import pandas as pd

# Load the processed trip-level data and aggregated driver-level data
processed_data_path = '../data/processed_data.csv'
aggregated_data_path = '../data/driver_aggregated_data.csv'

processed_df = pd.read_csv(processed_data_path)
aggregated_df = pd.read_csv(aggregated_data_path)

# Assign DriverId based on TripId (assuming that TripId is of the format 'T-x')
processed_df['DriverId'] = processed_df['TripId'].apply(lambda x: int(x.split('-')[1]))

# Group by DriverId in the processed data
grouped_processed_data = processed_df.groupby('DriverId')

# Define function to calculate the mode using pandas
def calculate_mode(series):
    return series.mode()[0]  # This returns the most frequent value

# Initialize lists to store mismatch results and comparison data for graphs
mismatches = []
comparison_data = []

# Iterate through each DriverId in the aggregated data
for index, row in aggregated_df.iterrows():
    driver_id = row['DriverId']
    
    # Get corresponding rows from processed data for the driver
    processed_group = grouped_processed_data.get_group(driver_id)
    
    # Calculate averages for the required columns in processed data
    avg_speed = processed_group['Speed(m/s)'].mean()
    avg_acceleration = processed_group['Acceleration(m/s^2)'].mean()
    avg_jerk = processed_group['Jerk(m/s^3)'].mean()
    avg_braking_intensity = processed_group['Braking_Intensity'].mean()
    avg_heading_change = processed_group['Heading_Change(degrees)'].mean()
    
    # Calculate modes for binary features (SASV and Speed Violation)
    sasv_mode = round(processed_group['SASV'].mean())  # SASV mode (since it's binary)
    speed_violation_mode = round(processed_group['Speed_Violation'].mean())  # Speed Violation mode (binary)
    
    # Calculate mode for driving category using pandas mode
    driving_category_mode = calculate_mode(processed_group['Driving_Category'].dropna())
    
    # Calculate average driving score from processed data
    avg_driving_score = round(processed_group['Driving_Score'].mean(), 2)
    
    # Compare calculated averages and modes with aggregated data
    comparison = {
        'DriverId': driver_id,
        'Processed_Speed': avg_speed,
        'Aggregated_Speed': row['Speed(m/s)_mean'],
        'Processed_Acceleration': avg_acceleration,
        'Aggregated_Acceleration': row['Acceleration(m/s^2)_mean'],
        'Processed_Jerk': avg_jerk,
        'Aggregated_Jerk': row['Jerk(m/s^3)_mean'],
        'Processed_Braking_Intensity': avg_braking_intensity,
        'Aggregated_Braking_Intensity': row['Braking_Intensity_mean'],
        'Processed_Heading_Change': avg_heading_change,
        'Aggregated_Heading_Change': row['Heading_Change(degrees)_mean'],
        'Processed_SASV': sasv_mode,
        'Aggregated_SASV': row['SASV_mode'],
        'Processed_Speed_Violation': speed_violation_mode,
        'Aggregated_Speed_Violation': row['Speed_Violation_mode'],
        'Processed_Driving_Score': avg_driving_score,
        'Aggregated_Driving_Score': round(row['Driving_Score'], 2),
        'Processed_Driving_Category': driving_category_mode,
        'Aggregated_Driving_Category': row['Driving_Category']
    }

    # Check for mismatches specifically in Driving Score and Driving Category
    if not all([
        comparison['Processed_Driving_Score'] == comparison['Aggregated_Driving_Score'],
        comparison['Processed_Driving_Category'] == comparison['Aggregated_Driving_Category']
    ]):
        mismatches.append(comparison)

# Create a dataframe from the mismatch data
mismatch_df = pd.DataFrame(mismatches)

# Display mismatch results specifically for Driving Score and Driving Category
if not mismatch_df.empty:
    print("Driving Score or Category mismatches found:")
    print(mismatch_df[['DriverId', 'Processed_Driving_Score', 'Aggregated_Driving_Score', 'Processed_Driving_Category', 'Aggregated_Driving_Category']])
else:
    print("No mismatches in Driving Score or Driving Category.")

# Save mismatch results if needed
mismatch_output_path = '../data/mismatch_results.csv'
mismatch_df.to_csv(mismatch_output_path, index=False)

print(f"Mismatches saved to {mismatch_output_path} if any.")


Driving Score or Category mismatches found:
    DriverId  Processed_Driving_Score  Aggregated_Driving_Score  \
0          1                    62.46                     58.88   
1          2                    61.84                     55.88   
2          3                    75.43                     81.58   
3          4                    50.33                     51.05   
4          5                    76.94                     82.29   
5          6                    71.69                     78.73   
6          7                    68.88                     62.70   
7          8                    71.17                     76.93   
8          9                    58.27                     55.78   
9         10                    76.73                     79.49   
10        11                    66.27                     62.14   
11        12                    65.91                     59.72   
12        13                    75.67                     81.39   
13        14      